Find large main-belt asteroids suitable for MRS fringe characterization.  <br />
This notebook modified after 'centaurs2019.ipynb' used for GTO planning.

M.Mueller@astro.rug.nl, 2021/09/27+

Find large MBAs observable with JWST during early routine science. 

User input: <br>
* Diameter range to be searched <br>
* minimum acceptable galactic latitude (reject asteroids while in Milky Way) <br>
* date range <br>

Step 1: query catalog of WISE-measured main-belt asteroid diameters for high-fidelity asteroid diameters within a user-defined range (here: defined by max acceptable angular diameter). <br>
Step 2: retrieve Horizons ephemerides for the asteroids found above within a user-defined date range, restrict to solar elongation within JWST's field of regard and |galactic latitude| > a user-defined value (reject asteroids in the Milky Way).<br>
Step 3: estimate (reflected + thermal) flux using WISE best-fit parameters, compare against sensitivity.  Output which objects are doable at what wavelength.  <br>

Caveat 1: not checking for ephemeris uncertainty at this point -- probably not an issue for these well-characterized objects, but would be straightforward given Horizons output. <br>
Caveat 2: not checking for motion rates, either.  MBAs should be fine, but check should be added if NEAs or comets ever are of interest. <br>
Caveat 3: after repeated runs, Horizons may block your URL for some time due to an excessive number of API calls.  Do we need to get whitelisted?  Is it a good idea to pre-calculate ephemerides rather than requesting them over and over again?
Caveat 4: some asteroids have multiple diameters, measured at different epochs.  This script will only consider the first value in the list.

In [1]:
from __future__ import print_function
import neatm
import astropy.units as u
from astropy.table import Table
from astropy.io import ascii
from scipy import interpolate
from urllib.request import urlopen
import numpy as np
import os
from astroquery.jplhorizons import Horizons

## Define global parameters

In [2]:
maxAngDiam = 0.03 * u.arcsec  # Is that a good criterion for MIRI point sources?
minAngDiam = maxAngDiam/1.5   # don't go very much smaller to keep high SNR
minAngDiam = maxAngDiam/1.09     # debugging
assumedDistance = 2*u.AU      # inner edge of main belt.  Further asteroids will appear smaller

maxDiam = (assumedDistance * maxAngDiam.to(u.radian).value).to(u.km).value
minDiam = (assumedDistance * minAngDiam.to(u.radian).value).to(u.km).value

In [3]:
startDate = '2022-07-01'
endDate   = '2022-07-11'
center='@JWST'
elongLimits = [85,135]

In [4]:
minGlxLat = 5 # in degree -- reject asteroids smack in the Milky Way

In [5]:
# fluxes to get SNR>300 in 40 groups * 4 dither positions, per dichroic at native spectral resolution
# See ETC notebook 89119 (which needs some attention, still)

minFlux = {}  
minFlux[10*u.micron]=250*u.mJy 
minFlux[20*u.micron]=1000*u.mJy 
wavelengths = [key for key in minFlux]

In [6]:
# epochs needed by astroquery.jplhorizons.Horizons
epochs={}
epochs['start']=startDate
epochs['stop']=endDate
epochs['step']='1d' # hard-wire: daily output

In [7]:
wiseCatalogFilename = os.path.join('neowise_diameters_albedos_V2_0', 'data', 'neowise_mainbelt.csv')
assert os.path.isfile( wiseCatalogFilename )
wise = ascii.read( wiseCatalogFilename )

In [8]:
wise.rename_column('col1', 'number')
wise.rename_column('col4', 'H')
wise.rename_column('col5', 'G')
wise.rename_column('col11', 'fitCode') # require DVBI for best-quality fits
wise.rename_column('col12', 'D')
wise.rename_column('col13', 'sigmaD')
wise.rename_column('col14', 'pV')
wise.rename_column('col15', 'sigmaPV')
wise.rename_column('col18', 'eta')
wise.rename_column('col19', 'sigmaEta')

In [9]:
idx = np.where( (wise['number']>0) & (wise['fitCode']=='DVBI') & (wise['D']<maxDiam) & (wise['D']>minDiam) )
candidates=wise[idx]

## Aux function

In [10]:
sunFluxFile='Rieke2008.fluxSunVega.txt'
assert os.path.isfile(sunFluxFile)
# Modified after Lenka's "Ref_Brightness.py" as of 2018/03/20: https://github.com/lenkaaaa49/Serendipitous-asteroid-observation
solar_flux_density=Table.read('Rieke2008.fluxSunVega.txt', guess=False,format='ascii.fixed_width',  delimiter=' ',
                       header_start=None, data_start=14,
                       col_starts=(0, 8, 17),
                      col_ends=(7, 16, 26),names=('Wavelength[micron]','Flux density of the Sun[W/m2/nm]',
                            'Flux density of Vega[W/m2/nm]'))
lambdasSun=solar_flux_density['Wavelength[micron]'].data
fluxSun = solar_flux_density['Flux density of the Sun[W/m2/nm]'].data
s = interpolate.InterpolatedUnivariateSpline(lambdasSun, fluxSun)
solarFluxes = np.array([(s(wav.to_value(u.micron))*u.Watt/u.meter**2/u.nm).to_value(u.mJy, equivalencies=u.spectral_density(wav)) for wav in wavelengths])*u.mJy

vSun=-26.74

def reflectedSunlight(v,relRefl=1.4): 
    # assume wavelengths defined globally....
    return solarFluxes*10**(-(v-vSun)/2.5)*relRefl

## Retrieve ephemerides, filter

In [11]:
fringeAsteroids={}
for line in candidates:
    # get ephemeris
    hor=Horizons( id='%s'%line['number'], location=center, epochs=epochs )
    try:
        asteroid = hor.ephemerides(solar_elongation=elongLimits) 
    except ValueError:
        print( line['number'], ", not in field of regard")
        continue
    observable = np.where( (abs(asteroid['GlxLat'])>minGlxLat) )
    if len(observable[0])>0:
        print (line['number'])
        fringeAsteroids[line['number']]=asteroid[observable]
    else:
        print ('.')

60 , not in field of regard
118
155 , not in field of regard
167 , not in field of regard
208 , not in field of regard
208 , not in field of regard
242
309
383 , not in field of regard
418 , not in field of regard
493
493
497
500 , not in field of regard
500 , not in field of regard
515
519 , not in field of regard
537 , not in field of regard
542 , not in field of regard
660
661 , not in field of regard
688 , not in field of regard
692 , not in field of regard
695
701 , not in field of regard
755 , not in field of regard
766 , not in field of regard
767
829 , not in field of regard
829 , not in field of regard
830 , not in field of regard
844 , not in field of regard
882 , not in field of regard
1008 , not in field of regard
1017 , not in field of regard
1024 , not in field of regard
1092 , not in field of regard
1096 , not in field of regard
1128 , not in field of regard
1194
1200
1284
1285 , not in field of regard
1337
1354 , not in field of regard
1369
1484 , not in field of regard

In [12]:
with open('fringeAsteroids.txt', 'w') as f:
    for obj in fringeAsteroids:
        eph = fringeAsteroids[obj]
        print (obj)
        f.write(str(obj)+ '\n')
        f.write('# H=%s\n'%eph['H'][0])  ## This is H,G from Horizons, not necessarily the same as used by WISE
        f.write('# G=%s\n'%eph['G'][0])
        for t, v, r, delta,alpha in zip(eph['datetime_jd'], eph['V'], eph['r'], eph['delta'], eph['alpha']):
            f.write('%.1f, %.2f, %.5f, %.5f, %.2f\n'%(t,v,r,delta,alpha))
        f.write('\n')

118
242
309
493
497
515
660
695
767
1194
1200
1284
1337
1369
1599
1838
3036


## Estimate fluxes, compare against sensitivity

In [13]:
brightEnough={}
print( "During the selected epoch, asteroid number N is observable for M days with sufficient brightness:")
print( "" )
for obj in fringeAsteroids.keys():
    brightEnough[obj]={}
    for wav in wavelengths:
        brightEnough[obj][wav]=[] # tuples: (datetime,flux)
    eph=fringeAsteroids[obj]
    idx=np.where(candidates['number']==obj)
    assert len(idx[0]) > 0
    
    h=candidates[idx]['H'][0]
    g=candidates[idx]['G'][0]
    eta=candidates[idx]['eta'][0]
    pv=candidates[idx]['pV'][0]
    for t, v, r, delta,alpha in zip(eph['datetime_str'], eph['V'], eph['r'], eph['delta'], eph['alpha']):
        thermal=neatm.neatm(h,g, alpha*u.deg, r*u.AU, delta*u.AU, wavelengths, eta, pv, mJy=True)
        thermal=np.array([x.to_value(u.mJy) for x in thermal])*u.mJy
        reflected=reflectedSunlight(v)
        total=reflected+thermal
        for l,f in zip(wavelengths, total):
            if f.to_value(u.mJy) > minFlux[l].to_value(u.mJy):
                brightEnough[obj][l].append((t,f))
    print (obj)
    for wav in wavelengths:
        print(wav, len(brightEnough[obj][wav]))
    print()


During the selected epoch, asteroid number N is observable for M days with sufficient brightness:

118
10.0 micron 11
20.0 micron 11

242
10.0 micron 8
20.0 micron 8

309
10.0 micron 2
20.0 micron 2

493
10.0 micron 0
20.0 micron 0

497
10.0 micron 4
20.0 micron 4

515
10.0 micron 0
20.0 micron 0

660
10.0 micron 11
20.0 micron 11

695
10.0 micron 11
20.0 micron 11

767
10.0 micron 11
20.0 micron 11

1194
10.0 micron 11
20.0 micron 11

1200
10.0 micron 11
20.0 micron 11

1284
10.0 micron 11
20.0 micron 11

1337
10.0 micron 11
20.0 micron 11

1369
10.0 micron 11
20.0 micron 11

1599
10.0 micron 10
20.0 micron 10

1838
10.0 micron 8
20.0 micron 4

3036
10.0 micron 8
20.0 micron 8

